In [ ]:
# 配置日志
import logging
import logging.config
import yaml
with open('.conf/logging.yml', 'r') as f_conf:
    dict_conf = yaml.safe_load(f_conf)
logging.config.dictConfig(dict_conf)

filepath='logs/'
del_list = os.listdir(filepath)
for f in del_list:
    file_path = os.path.join(filepath, f)
    if os.path.isfile(file_path):
        os.remove(file_path)


In [ ]:
from DPC import DPCLink
from load import *
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import itertools

# 读取文件为迭代器, 可指定读取数量
dataset_path = 'dataset/Geolife Trajectories 1.3/Data/001/'
dataset = enumerate(itertools.islice(LoadG(dataset_path), 0, None))

# 读取数据
ls = []
dfs = pd.DataFrame()
for i, df in dataset:
    df['trackID'] = i + 1
    dfs = pd.concat([dfs, df], ignore_index=True)
dfs.reset_index(drop=True)
dfs.plot.scatter(x='lng', y='lat', s=0.0001)

# 可视化轨迹在地图上
m = folium.Map(location=[39.984094, 116.319236], zoom_start=16)
ls = []
dataset = enumerate(itertools.islice(LoadG(dataset_path), 0, None))
for i, df in dataset:
    l = [[lat, lng]for [lat, lng, *_] in df.values]
    folium.PolyLine(l, weight=0.4).add_to(m)
    ls.extend(l)
HeatMap(ls).add_to(m)
m.save('map/df.html')

# 初始化类, 后续运算都通过此类操作
dpc = DPCLink(dfs)

dpc.calDis()


In [ ]:

dpc.calDc(p=0.01)
dpc.calRho()

dpc.calDel()
dpc.calGam()
dpc.getCen(10)

dpc.cluster()


In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=500)
dpc.df.plot.scatter(x='lng', y='lat', c='clusterID', colormap='tab20', s=0.2,alpha=0.3, linewidths=0, ax=ax)
dpc.df.iloc[dpc.centers].plot.scatter(x='lng', y='lat', c='red', s=0.1, ax=ax)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

m = folium.Map(location=[39.984094, 116.319236], zoom_start=16)
ls = []
dataset = enumerate(itertools.islice(LoadG(dataset_path), 0, 50))
for i, df in dataset:
    l = [[lat, lng]for [lat, lng, *_] in df.values]
    folium.PolyLine(l, weight=0.4).add_to(m)
    ls.extend(l)
HeatMap(ls).add_to(m)

for p in dpc.centers:
    folium.Marker([dpc.df.loc[p, 'lat'], dpc.df.loc[p, 'lng']]).add_to(m)
m.save('map/df.html')


In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=500)
# dpc.df.plot.scatter(x='lng', y='lat', c='clusterID', colormap='tab20', s=0.5, edgecolors='none', ax=ax)
dfs.plot.scatter(x='lng', y='lat', s=0.1, alpha=0.1, linewidths=0, ax=ax)

p = dpc.df.iloc[6382]
ax.scatter(p.lng, p.lat, c='red', s=0.1)
# dpc.df.iloc[dpc.centers].plot.scatter(x='lng', y='lat', s=0.1, c='red', ax=ax)

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
